In [31]:
#inline('sqrt(2)*an.*sin(pl*x)','x','an','pl'); # divide by L !!!!
q = (lambda x, an, pl:  numpy.sqrt(2)*an*numpy.sin(pl*x))
q(math.pi,1,0.5)

1.4142135623730951

In [66]:
import numpy, math

l=0.6      # length in m
b=0.06     # width beam also width piezoelement
h=0.0034   # width and heigh in m
rho=2700   # Alu kg/m^3
E=65e9     # Pa                      
nu=0.34    # poisson's ratio alu
I=(b*h**3)/12   # Interia
Ar=b*h     # section surface
M=l*rho*Ar #  mass beam
niu=0.03   # damping ratio (zeta)

Nmodes=1;


an = 1.000; # is for consistency of notation
def bara(cond, A, I, L, E=1.9995E11, rho=7862.3, niu=0):
    ing = numpy.arange(1, 31) # index valori modale
    xmed = L/2;
    
    pl=(ing)*math.pi;
    b = { }
    b["an"] = numpy.ones(len(pl));    # don't need it here !
    
    b["yp"] = (lambda x, an, pl:  numpy.sqrt(2)*an*numpy.sin(pl*x)) # divide by L !!!!
    b["yp1"] = (lambda x, an, pl:  numpy.sqrt(2)*an*pl*numpy.cos(pl*x))
    b["yp2"] = (lambda x, an, pl:  -numpy.sqrt(2)*an*pl*pl*numpy.sin(pl*x));
    b["yp3"] = (lambda x, an, pl:  -numpy.sqrt(2)*an*pl*pl*pl*numpy.cos(pl*x));
      
         
    b["sag"] = L**3/48/E/I;
    b["defl"] = (lambda x, L, EI:   -(x**4 - 2*L*x**3 + L**3*x)/24/EI)
#        b["Iyp"] = inline('(1-cos(bl)).*(an./bl)','an','bl');
        
    b["pl"] = pl;
    b["om2"] = (E*I/rho/A)*(pl/L)**4;          # omega squared
    b["vcr"] = numpy.sqrt(E*I/rho/A)*(pl[1]/L);
    b["roA"] = rho*A;
    b["EI"] = E*I;
    b["niu"] = 2*numpy.sqrt(b["om2"])*niu;              # Damping factor zeta
    b["L"] = L;
    b["NormX"] = L;                            # norma lui X, ||X|| = L sau L/2
    b["xmed"] = xmed;
    b["EIroA"] = E*I/rho/A;
    b["Mass"] = rho*A*L;
    b["I"] = I;
    b["A"] = A;
    return b
    
b = bara('sup',Ar,I,l,E,rho,niu);
t0 = 0
tfinal=1.5;
x0 = numpy.zeros(Nmodes*2);
v=0.55; #speed ball
ml=0.5; # mass of the ball



In [152]:
#ssfmodelMLOL(t,x,b,v,Nmodes,ml)
def ssfmodelMLOL(t, x, b, v, Nmodes, mass_Load):
    xa = v*t;  # this is not an ordanary differential equation
    L = b["L"];
    pl = b["pl"][0:Nmodes]/L; # 
    an = b["an"][0:Nmodes];
    Mass = b["roA"]*L*numpy.eye(Nmodes);
    K = b["EI"]*L*numpy.diag(pl**4);
    D = numpy.diag(b["niu"][0:Nmodes])*Mass;

    dM = numpy.matmul(mass_Load*b["yp"](v*t,an,pl), b["yp"](v*t,an,pl))
    dD= 2*numpy.matmul(mass_Load*v*b["yp"](v*t,an,pl), b["yp1"](v*t,an,pl))
    dK=mass_Load*numpy.matmul(v**2*b["yp"](v*t,an,pl), b["yp2"](v*t,an,pl))

    # added damping and stiffness shaker
    Ka=numpy.matmul(12000*b["yp"](0.5,an,pl), b["yp"](0.5,an,pl));
    Da=numpy.matmul(50*b["yp"](0.5,an,pl), b["yp"](0.5,an,pl));

    if xa<=L:
        As = numpy.matrix([[0, 1], 
                           [-(Mass+dM)*1/(K+dK+Ka), -(Mass+dM)*1/(D+dD+Da)]])
        BF = numpy.matrix([[0], [-(Mass+dM)*1/b["yp"](v*t,an,pl)]]);

        xdot = As*(numpy.array([x]).T)+BF*(mass_Load*9.81);

    elif xa>L: 
        As = numpy.matrix([[0, 1], 
                              [-(Mass)*1/(K+Ka), -(Mass)*1/(D+Da)]])

        xdot = As*(numpy.array([x]).T);
    return numpy.array(list(map(float, xdot)))  # we need to abolish numpy.matrix

ssfmodelMLOL(1, numpy.array([1,2]), b, v, Nmodes, ml)



array([ 2.        , -5.35674396])

In [131]:
x = numpy.array([1,2])
#xdot = As@numpy.array([x]) + (BF*(mass_Load*9.81))
xdot
As*(numpy.array([x]).T)

matrix([[ 2.        ],
        [-0.03040028]])

In [123]:
numpy.matrix([[0,1],[0,0]])*(numpy.array([[4,5]]).T)

matrix([[5],
        [0]])

In [124]:
xdot

matrix([[ 2.        , -5.35674396]])

In [80]:
#if xa<=L:
As = numpy.matrix([[1, 1], [-(Mass+dM)*1/(K+dK+Ka), -(Mass+dM)*1/(D+dD+Da)]])
As

matrix([[ 1.00000000e+00,  1.00000000e+00],
        [-3.37981831e-05, -1.51832419e-02]])

In [154]:
import scipy.integrate

ssfmodelMLOL(t,x,b,v,Nmodes,ml)



r = scipy.integrate.ode(ssfmodelMLOL).set_integrator('vode', method='bdf', order=15)
r.set_initial_value(x0, t0).set_f_params(b,v,Nmodes,ml)

#t1 = 10
#dt = 1
#while r.successful() and r.t < t1:
#    r.integrate(r.t+dt)

#[t2,x2] = ode15s(@(t,x) ssfmodelMLOL(t,x,b,v,Nmodes,ml), t0:tfinal/500:tfinal,x0);
#w2 = deflection(x2,b,[0.15 0.25 0.35])*1000; # deflection in mm
#plot(t2,w2)

In [155]:
r.integrate(0.01)

/home/julian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in true_divide
/home/julian/anaconda3/lib/python3.6/site-packages/scipy/integrate/_ode.py:1009: UserWarning: vode: Repeated convergence failures. (Perhaps bad Jacobian supplied or wrong choice of MF or tolerances.)
  self.messages.get(istate, unexpected_istate_msg)))


array([ 0., nan])